In [ ]:
"""
演示怎么在训练好的模型上重新加载并运行。
"""


import pandas as pd
from numpy import zeros
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
import re
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Dense, LSTM, Bidirectional
from tensorflow.keras import regularizers
from tensorflow.keras.utils import plot_model

first_train_line = 300
second_train_line = 600

# 1. 读取数据
pima_data = pd.read_csv("./input_data/diabetes.csv")
pima_data.info()
first_train_data = pima_data[0:first_train_line]
second_train_data = pima_data[first_train_line:second_train_line]
test_data = pima_data[second_train_line:]

X_1_train = first_train_data.iloc[:, 0:8]
Y_1_train = first_train_data.iloc[:, 8]
X_2_train = second_train_data.iloc[:, 0:8]
Y_2_train = second_train_data.iloc[:, 8]
X_test = test_data.iloc[:, 0:8]
Y_test = test_data.iloc[:, 8]

np_X_1_train = X_1_train.to_numpy()
np_Y_1_train = Y_1_train.to_numpy()
np_X_2_train = X_2_train.to_numpy()
np_Y_2_train = Y_2_train.to_numpy()
np_X_test = X_test.to_numpy()
np_Y_test = Y_test.to_numpy()

model = Sequential()
model.add(Dense(20, input_shape=(8, )))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
plot_model(model, to_file="./model_info/pima_model_structure.png" , show_shapes=True)

# 2. 存取模型。在val_acc最好的情况下把模型存取。
# 这种方式把模型所有的信息都存下来了，包括模型结构，每层的权重，最有函数的选择等。
model_file="./model_info/pima_model"
checkpoint = ModelCheckpoint(model_file, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
history = model.fit(np_X_1_train, np_Y_1_train, batch_size=16, epochs=20, verbose=1, validation_split=0.2, callbacks=callbacks_list)
score, accuracy = model.evaluate(np_X_test, np_Y_test, verbose=0)
print("test score : {} test acc : {}".format(score, accuracy))

# 3. load模型继续训练。
loaded_model = load_model(model_file)
score, accuracy = loaded_model.evaluate(np_X_test, np_Y_test, verbose=0)
print("test score : {} test acc : {}".format(score, accuracy))
history2 = loaded_model.fit(np_X_2_train, np_Y_2_train, batch_size=16, epochs=20, verbose=1, validation_split=0.2)
score, accuracy = loaded_model.evaluate(np_X_test, np_Y_test, verbose=0)
print("test score : {} test acc : {}".format(score, accuracy))
